# Определение токсичности комментариев

Данный проект состоит в обучении модели классификации комментариев на позитивные и негативные (токсичные). В нашем распоряжении набор комментариев с разметкой о токсичности. Необходимо построить модель со значением метрики качества F1 не меньше 0.75. 

## 1. Загрузка и подготовка данных

Сперва импортируем необходимые в дальнейшей работе библиотеки:

In [9]:
import pandas as pd
import nltk
nltk.download('wordnet')
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
import re
#!pip install tqdm
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
nltk.download('stopwords') 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
#!pip install lightgbm
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nikit\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nikit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nikit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Теперь загрузим набор данных:

In [10]:
try:
    comments = pd.read_csv('toxic_comments.csv')
except:
    comments = pd.read_csv('/datasets/toxic_comments.csv')

Ознакомимся с данными:

In [11]:
comments.head(3)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0


Заметим столбец Unnamed: 0, дублирующий индекс. Удалим его:

In [12]:
comments = comments.drop('Unnamed: 0', axis=1)

Кроме того, оценим размер таблицы:

In [13]:
comments.shape

(159292, 2)

Как видим, мы имеем дело с 159292 текстами.

Напишем функцию для **лемматизации** текста. Так как комментарии у нас на английском языке, будем пользоваться средставми библиотеки **nltk**:

In [14]:
lemmatizer = WordNetLemmatizer()

def lemmatize(text):
    lemm_list = nltk.word_tokenize(text)
    lemm_text = ' '.join([lemmatizer.lemmatize(w) for w in lemm_list])   
    return lemm_text

Также напишем функцию для **очистки текста** от лишних символов:

In [15]:
def clear_text(text):
    lemm_list = re.sub(r'[^a-zA-z]', ' ', text)
    lemm_list_split = lemm_list.split() 
    lemm_text = " ".join(lemm_list_split)
    return lemm_text

Создадим в датафрейме новую колонку, содержащую исходные тексты, к которым были применены написанные нами функции. С помощью средств библиотеки tqdm будем отслеживать прогресс работы:

In [16]:
tqdm.pandas()
comments['lemm_text'] = comments['text'].progress_apply(clear_text).progress_apply(lemmatize)

100%|██████████| 159292/159292 [10:28<00:00, 253.34it/s]


Приведем ее к нижнему регистру:

In [17]:
comments['lemm_text'] = comments['lemm_text'].str.lower()

Теперь рассчитаем **TF-IDF**. Сперва поделим данные на выборки:

In [18]:
features = comments['lemm_text']
target = comments['toxic']

features_train, features_test, target_train, target_test = train_test_split(features, target, 
                                                          test_size=0.25, random_state=42, stratify=target)

Теперь рассчитаем TF-IDF:

In [19]:
stop_words = set(stopwords.words('english')) 
count_tf_idf = TfidfVectorizer(stop_words=stop_words)

In [20]:
features_train = count_tf_idf.fit_transform(features_train)
features_test = count_tf_idf.transform(features_test)

## 2. Обучение моделей

Начнем с модели **логистической регрессии**:

In [21]:
model = LogisticRegression(class_weight='balanced', max_iter=110)

Проведем **кросс-валидацию** модели:

In [22]:
scores = cross_val_score(model, features_train, target_train, scoring='f1')

C:\Users\nikit\anaconda3\envs\ds_practicum_env\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [23]:
print(scores)
print(scores.mean())

[0.74732571 0.74759793 0.74670088 0.74981604 0.74478686]
0.7472454845222967


**Среднее значение метрики** (с округлением до сотых) составляет искомый порог - **0.75**. 

Проведем **гридсерч**:

In [24]:
parameters = {
    'C':[0.5, 1, 10],
    'max_iter': range(100, 1000, 100),
}

grid = GridSearchCV(model, parameters, scoring='f1', n_jobs = -1, verbose=3)

In [25]:
grid.fit(features_train, target_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


C:\Users\nikit\anaconda3\envs\ds_practicum_env\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(estimator=LogisticRegression(class_weight='balanced',
                                          max_iter=110),
             n_jobs=-1,
             param_grid={'C': [0.5, 1, 10], 'max_iter': range(100, 1000, 100)},
             scoring='f1', verbose=3)

In [26]:
print(grid.best_score_)    
print(grid.best_params_)

0.7635137660962845
{'C': 10, 'max_iter': 200}


<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Супер! Молодец, что по подбирал параметры
</div>

Модель с гиперпараметрами 'C': 10, 'max_iter': 200 получила **значение метрики** в **0.76**, что удовлетворяет порогу. 

Обучим модель **LGBMClassifier**:

In [27]:
model = LGBMClassifier(
    boosting_type='gbdt',
    num_leaves=31,
    max_depth=-1,
    learning_rate=0.1,
    n_estimators=100,
    objective='binary',
    min_split_gain=0.0,
    min_child_samples=20,
    subsample=1.0,
    subsample_freq=0,
    colsample_bytree=1.0,
    reg_alpha=0.0,
    reg_lambda=0.0,
    random_state=None
)

In [28]:
scores = cross_val_score(model, features_train, target_train, 
                         scoring='f1', n_jobs = -1, verbose=3)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  8.1min remaining: 12.2min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 11.7min finished


In [29]:
print(scores)
print(scores.mean())

[0.73965351 0.74034491 0.74692993 0.74454678 0.73791596]
0.7418782178536923


**Значение метрики** f1 составило **0.74**. Попробуем поднять его с помощью **гридсерча**:

In [30]:
parameters = {
              'max_depth': range(10, 31, 10),
              'learning_rate': [0.1, 0.2],
              'n_estimators': range(40, 121, 40)
}

grid = GridSearchCV(model, parameters, scoring='f1', n_jobs = -1, verbose=3)

In [31]:
grid.fit(features_train, target_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(estimator=LGBMClassifier(objective='binary'), n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.2],
                         'max_depth': range(10, 31, 10),
                         'n_estimators': range(40, 121, 40)},
             scoring='f1', verbose=3)

In [32]:
print(grid.best_score_)
print(grid.best_params_)

0.7636820785690078
{'learning_rate': 0.2, 'max_depth': 30, 'n_estimators': 120}


Лучшее **значение метрики** составило **0.76** при гиперпараметрах 'learning_rate': 0.2, 'max_depth': 30, 'n_estimators': 120. Такое значение также проходит порог в 0.75. Значение метрики без округление несколько выше, чем значения для модели логистической регрессии (хоть и очень незначительно). Поэтому выберем ее для **проверки на тестовых данных**. 

Создадим новую модель по результатам гридсерча:

In [33]:
model = LGBMClassifier(
    boosting_type='gbdt',
    num_leaves=31,
    max_depth=30,
    learning_rate=0.2,
    n_estimators=120,
    objective='binary',
    min_split_gain=0.0,
    min_child_samples=20,
    subsample=1.0,
    subsample_freq=0,
    colsample_bytree=1.0,
    reg_alpha=0.0,
    reg_lambda=0.0,
    random_state=None
)

**Обучим модель** на тренировочной выборке:

In [34]:
model.fit(features_train, target_train)

LGBMClassifier(learning_rate=0.2, max_depth=30, n_estimators=120,
               objective='binary')

**Предскажем** тестовые данные:

In [35]:
target_predicted = model.predict(features_test)

**Посчитаем метрику f1**:

In [36]:
f1_score(target_test, target_predicted)

0.7676767676767675

Видим, что значение метрики составило **0.77**, что удовлетворяет порогу в 0.75. **Обучение можем считать успешным**. 

В ходе исследования мы провели обработку набора данных о токсичности комментариев: провели **лемматизацию**, использовали **регулярные выражения** для фильтрации нужных слов в тексте, вычислили **TF-IDF**. После предобработки данные были разделены на обучающую и тестовую выборку.

Затем мы провели **обучение** моделей **логистической регрессии** и модели **LGBMClassifier**. В ходе обучения мы осуществляли **кросс-валидацию** и **гридсерч** для поиска наилучшей конфигурации моделей. По результатам подбора гиперпараметров получили две модели, практически идентично оцениваемые метрикой **f1**: **0.76348 для регрессии и 0.76368 для LGBMClassifier**. В итоге, для **тестирования** была выбрана модель **LGBMClassifier**, показавшая *небольшое* превосходство по этой метрике.

По итогам **проверки** модели на **тестовых** данных мы получили значение метрики в **0.76768**, что укладывается в техническое задание. В связи с чем, можем считать, что **наше исследование достигло своих целей**.